In [1]:
########################################
#### ADD geojson to perimeter table ####
########################################
import shapely
import datetime
import geopandas as gpd
import pandas as pd
import uuid
import os

In [159]:
def create_landscape(df, index):
    
    description = df.loc[index,'description']
    print(f'Creating landscape for {description}')
    gdf = gpd.read_file(df.loc[index,'filepath'])
    geom_5070 = gdf['geometry'].iloc[0]
    bounds = geom_5070.bounds

    ulx = bounds[0]-10000
    uly = bounds[3]+10000
    lrx = bounds[2]+10000
    lry=  bounds[1]-10000

    fname_lst = {'density': 'US_140CBD_12052016/Grid/us_140cbd', 
                 'base': 'US_140CBH_12052016/Grid/us_140cbh', 
                 'cover': 'US_140CC_12052016/Grid/us_140cc', 
                 'height': 'US_140CH_12052016/Grid/us_140ch', 
                 'fuel': 'US_140FBFM40_20180618/Grid/us_140fbfm40', 
                 'aspect': 'Aspect/Grid/us_asp', 
                 'elevation': 'DEM_Elevation/Grid/us_dem', 
                 'slope': 'Slope/Grid/us_slp'}
    type_lst = {'density': 'cbd',
                'base': 'cbh',
                'cover': 'cc',
                'height': 'ch',
                'fuel': 'fuel',   # fbfm40
                'aspect': 'aspect',
                'elevation': 'elevation', #dem
                'slope': 'slope'}

    from_folder = os.path.join('/data', 'firemap', 'landfire', 'mosaic')
    to_folder = '/home/tcaglar/farsite/inputs/landscapes/'

    # Create the asc files
    ascpath_lst = {}
    for (key, fname) in fname_lst.items():
        ascpath_lst[key] = f'{os.path.join(to_folder, description)}-{key}.asc'
        os.system(f'gdal_translate -of AAIGrid -a_nodata -32768 -projwin {ulx} {uly} {lrx} {lry} {os.path.join(from_folder, fname)} {ascpath_lst[key]}')
        
    lcppath = f'{os.path.join(to_folder, description)}'
    lcpmakepath = '/home/tcaglar/farsite/src/lcpmake'

    base_command = f'{lcpmakepath} -latitude {gdf.to_crs(epsg=4326)["geometry"].iloc[0].centroid.y} -landscape {lcppath}'
    run_command = base_command
    for (key, ascpath) in ascpath_lst.items():
        run_command += f' -{key} {ascpath}'

    os.system(run_command)
    
    os.system(f"rm {os.path.join(to_folder, '*asc')}  {os.path.join(to_folder, '*prj')} {os.path.join(to_folder, '*xml')}")
    
    return to_folder

def append_table(df, gdf, description):

    input_folder = f'/home/tcaglar/farsite/inputs/{description}_ignite/'
    if not os.path.exists(input_folder):
        os.mkdir(input_folder)
    gdfSub = gdf[gdf['description'] == description]

    for (idx, gdfrow) in gdfSub.iterrows():
        geom = gdfrow['geometry']
        objectid = gdfrow['objectid']

        fpath = f'{input_folder}{description}_{objectid}.shp'

        gpd.GeoDataFrame(data={'FID': [0]}, geometry=[geom]).set_crs(epsg=5070).to_file(fpath)
        gdfSub.loc[idx, 'filepath'] = fpath

    df = pd.concat([df, gdfSub[['filetype', 'objectid', 'filepath', 'datetime', 'description']]])

    # # Add landscape
    index = df[df['description'] == description].index[0]
    to_folder = create_landscape(df, index) 

    lcpidx = uuid.uuid4().hex
    df.loc[lcpidx, 'filetype'] = 'Landscape'
    df.loc[lcpidx, 'filepath'] = os.path.join(to_folder, f'{description}.lcp')
    df.loc[lcpidx, 'description'] = description

    return df


In [160]:
# Choose the geojson file
gdf = gpd.read_file('/home/tcaglar/data/bridge-river.geojson')

gdf['datetime'] = gdf['created_date'].apply(lambda date: datetime.datetime.fromtimestamp(date/1e3))

gdf = gdf.to_crs(epsg=5070)

gdf['index'] = [uuid.uuid4().hex for i in range(len(gdf))]

gdf = gdf.set_index('index')
gdf['filetype'] = 'Ignition'

gdf.loc[(gdf['source'] == 'River'), 'description'] = 'River2021'
gdf.loc[(gdf['source'] == 'Bridge'), 'description'] = 'Bridge2021'


for dt in gdf['datetime'].unique():
    gdfCombine = gdf.loc[gdf['datetime'] == dt]
    if len(gdfCombine) == 1:
        gdf.loc[(gdf['datetime'] == dt), 'shape'] = gdfCombine['geometry'].to_wkb()
        continue
    
    polygon_list = []
    for (idx, row) in gdfCombine.iterrows():
        polygon_list.append(row['geometry'])
    
    mpolygon = shapely.geometry.MultiPolygon(polygons=polygon_list)
    
    gdf.loc[(gdfCombine.index[0]), 'shape'] = mpolygon.to_wkb()
    
gdf = gdf.dropna(subset=['shape'])
gdf['geometry'] = gpd.GeoSeries.from_wkb(gdf['shape'])
gdf = gdf.set_crs(epsg=5070)

In [164]:
df = pd.read_pickle('/home/tcaglar/data/test_table.pkl')

append_descs = gdf['description'].unique()
for desc in append_descs:
    df = append_table(df, gdf, desc)

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


Creating landscape for River2021
Creating landscape for Bridge2021


In [168]:
df.to_pickle('/home/tcaglar/data/test_table_new.pkl')

,filetype,objectid,filepath,datetime,description
f23196b034474744bdca7df94b13e0f7,Ignition,21231,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 19:37:11,Maria2019
9f82e870591748a9a8a01346d174f2a1,Ignition,21232,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 19:58:22,Maria2019
1a6eb30ee33742e18fa6bb26678b5077,Ignition,21234,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 20:31:33,Maria2019
911f9b001c054b74a6ba70958f5f3c93,Ignition,21235,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 20:56:44,Maria2019
7957c2e604554199ba60c5b13aff2f40,Ignition,21236,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 21:22:55,Maria2019
714f0cf8d475462a97c651642a3d2525,Ignition,21237,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 23:31:00,Maria2019
628d7730c0f149f898695d37a1350882,Ignition,21238,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-10-31 23:48:00,Maria2019
746e4f286cba4af4a89362b548f2c255,Ignition,21239,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-11-01 00:29:00,Maria2019
ec72de65d1934e0ab74ff46ffba1ae1b,Ignition,21240,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-11-01 00:54:00,Maria2019
15dc4b77768f4921be77c65ae9a6b9d4,Ignition,21241,/home/tcaglar/farsite/inputs/maria_ignite/mari...,2019-11-01 01:12:00,Maria2019
